<a href="https://colab.research.google.com/github/jellyho/AI_Learning_jellyho/blob/main/10_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2021-10-03 05:21:09--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  70.3MB/s    in 1.1s    

2021-10-03 05:21:10 (70.3 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [4]:
import tarfile
with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
  tar.extractall()

In [5]:
pip install pyprind

In [6]:
import pyprind
import pandas as pd
import os

basepath = '/content/aclImdb'

labels = { 'pos':1, 'neg':0 }
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
  for l in ('pos', 'neg'):
    path = os.path.join(basepath, s, l)
    for file in sorted(os.listdir(path)):
      with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
        txt = infile.read()
      df = df.append([[txt, labels[l]]], ignore_index=True)
      pbar.update()


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:33


In [7]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('move_data.csv', index=False, encoding='utf-8')

In [19]:
df = pd.read_csv('/content/drive/MyDrive/move_data.csv', encoding='utf-8')
df.columns = ['review', 'sentiment']

In [16]:
df.shape

(50000, 2)

###데이터를 다운로드 받고 csv 파일 생성까지 완료!

In [21]:
import re
def preprocessor(text):
  text = re.sub('<[^>]*', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
  text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
  return text

In [22]:
df['review'] = df['review'].apply(preprocessor)

In [23]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


def tokenizer(text):
  return text.split()

tfidf = TfidfVectorizer(ngram_range=(1,1), tokenizer=tokenizer, stop_words=None)

model = Pipeline([('tfidf', tfidf),('lg', LogisticRegression(solver='liblinear', random_state=0, penalty='l2', C=10.0))])

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...b\\w\\w+\\b',
                                 tokenizer=<function tokenizer at 0x7ffb427e4050>,
                                 use_idf=True, vocabulary=None)),
                ('lg',
                 LogisticRegression(C=10.0, class_weight=None, dual=False,
                                    fit_in

In [24]:
model.score(X_test, y_test)

0.89856

In [43]:
import pickle

pickle.dump(model, open(os.path.join('/content/drive/MyDrive/pickles','model.pkl'), 'wb'), protocol=4)

In [63]:
testlist = ['Its just that kind of movie.', 'so funny!', 'Its a movie I dont want to watch again.']
test = np.array(testlist)

result = model.predict(test)
prob = np.max(model.predict_proba(test)*100,axis=1)

print(result)
print(prob)

[0 1 0]
[97.98947845 65.53776722 82.97968507]
